In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import numpy as np
import numpy.linalg as la
from scipy.integrate import RK45

import sympy as sym
# Heaviside numerics workaround
sympy_modules = [{'Heaviside': lambda x: np.heaviside(x,0.5)}, 'numpy']

from table_maker import *
from functools import partial, reduce
from itertools import *
from math import ceil

import pickle

def cos_bell(x, center=0, width=2*np.pi, height=1):
    return (np.cos((x-center)/width*2*np.pi)+1)/2*height * np.heaviside(x-center+width/2,0) * np.heaviside(-x+center+width/2,0)

import collections.abc
def collect_eqn(expr, terms):
    if not isinstance(terms, collections.abc.Container):
        term = terms # isn't a list or tuple
        left, right = 0,0
        for my_expr in expr.args:
            has_term = False
            for my_term in sym.postorder_traversal(my_expr):
                if term == my_term:
                    has_term = True
                    break
            if has_term:
                left += my_expr
            else:
                right -= my_expr
        return sym.Eq(left, right, evaluate=False)
    else: # isn't a list or tuple
        left_total, right_total = 0, expr
        for term in terms:
            left, right = collect_eqn(right_total, term).args
            left_total += left
            right_total = -right
        return sym.Eq(left_total, right_total, evaluate=False)
    
def my_print(expr):
    global eviron_print_tex
    if eviron_print_tex:
        print('$$' + sym.latex(expr) + '$$')
    else:
        display(expr)
        
def batch_print(expr_list, punctuation='.', return_string=False):
    global eviron_print_tex
    if eviron_print_tex:
        print_str = '$$\\begin{align*}\n'
        for expr in expr_list:
            print_str += '\t' + sym.latex(expr).replace('=','&=') + '\\\\\n'
        # remove last new-line and add ending puncuation
        print_str = print_str[:-3] + punctuation + '\n'
        print_str += '\\end{align*}$$'
        if return_string:
            return print_str
        else:
            print(print_str)
    else:
        for expr in expr_list:
            display(expr)
            
def py_code(expr, replace_numpy=True, replace_unicode=True, numpy_package='np.'):
    code = sym.ccode(expr)
    
    if replace_unicode:
        unicode_replacements = [
            ('\\alpha', 'α'),
            ('\\beta', 'β'),
            ('\\gamma', 'γ'),
            ('\\Gamma', 'Γ'),
            ('\\delta', 'δ'),
            ('\\Delta', 'Δ'),
            ('\\epsilon', 'ϵ'),
            ('\\zeta', 'ζ'),
            ('\\eta', 'η'),
            ('\\theta', 'θ'),
            ('\\xi', 'ξ'),
            ('\\mu', 'μ')
        ]
        for unicode, symbol in unicode_replacements:
            code = code.replace(unicode, symbol)
    
    if replace_numpy:
        numpy_replacements = [
            ('exp', numpy_package + 'exp'),
            ('log', numpy_package + 'log'),
            ('pow', '( lambda base, exponent: base**exponent )'),
            ('Heaviside', '(lambda input: np.heaviside(input,0.5))')
        ]
        for c_func, np_func in numpy_replacements:
            code = code.replace(c_func, np_func)
        
    return code

In [2]:
import matplotlib
matplotlib.rcParams.update({'font.size': 24})

In [3]:
eviron_print_tex = False

μ, α, γ, θ, c, Δ, λ = sym.symbols('\\mu \\alpha \\gamma, \\theta, c \\Delta \\lambda', real=True, positive = True)

ξ = sym.symbols('\\xi', real=True)

U = sym.Function('U', real=True)(ξ)
A = sym.Function('A', real=True)(ξ)

LU = lambda f: c*μ*f.diff(ξ) - f
LA = lambda f: c*α*f.diff(ξ) - f

F_shared = (sym.exp(Δ)-1)/(sym.exp(λ)-1)/2 * ( sym.exp(-ξ-Δ) + sym.exp(ξ)) - sym.exp(-ξ-Δ)/2
F_left = F_shared + 1 - sym.exp(-ξ)/2
F_right = F_shared + sym.exp(-ξ)/2

F_left = F_left.expand().collect(sym.exp(ξ))
F_right = F_right.expand().collect(sym.exp(ξ))

G_left = 0
G_right = γ

In [4]:
# left
U_hom = sym.dsolve(LU(U))
AL, BL, CL = sym.symbols('A_L, B_L, C_L', real=True)
form = AL + BL*sym.exp(ξ) + CL*sym.exp(-ξ)

display(F_left)
display(LU(form).expand().collect(sym.exp(ξ)))

L_particular = form.subs({ AL: -1, BL: F_left.coeff(sym.exp(ξ))/(μ*c-1), CL: F_left.coeff(sym.exp(-ξ))/(μ*c-1) })
display(L_particular)

(exp(\Delta)/(2*exp(\lambda) - 2) - 1/(2*exp(\lambda) - 2))*exp(\xi) + (-1/2 - exp(-\Delta)/2 + 1/(2*exp(\lambda) - 2) - 1/(2*exp(\Delta)*exp(\lambda) - 2*exp(\Delta)))*exp(-\xi) + 1

-A_L + (B_L*\mu*c - B_L)*exp(\xi) + (-C_L*\mu*c - C_L)*exp(-\xi)

-1 + (exp(\Delta)/(2*exp(\lambda) - 2) - 1/(2*exp(\lambda) - 2))*exp(\xi)/(\mu*c - 1) + (-1/2 - exp(-\Delta)/2 + 1/(2*exp(\lambda) - 2) - 1/(2*exp(\Delta)*exp(\lambda) - 2*exp(\Delta)))*exp(-\xi)/(\mu*c - 1)

In [5]:
(LU(L_particular) - F_left).expand().simplify()

\mu*c*(-2*exp(\Delta) + exp(\lambda) + exp(\Delta + \lambda))*exp(-\Delta - \xi)/(\mu*c*exp(\lambda) - \mu*c - exp(\lambda) + 1)